In [ ]:
from z3 import *
from itertools import combinations
import time
import numpy as np

# Aux functions

def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool("s_{}_{}".format(name,i)) for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one(bool_vars, name):
    return And(at_most_one(bool_vars, name),at_least_one(bool_vars))

def exactly_zero(bool_vars):
    return Not(Or(bool_vars))


# Instance 
class Instance(object):
    width = 0
    n = 0
    dimensions = []
    def __init__(self, width, n, dimensions):
        self.width = width
        self.n = n
        self.dimensions = dimensions


# Read instances: 
def read_file(file_name):
    dimensions = []
    with open(file_name) as f:
        width = int(f.readline())                 # Width of the plate
        n = int(f.readline())                     # Number of blocks
        while True:
            line = f.readline()
            if not line: 
                break
            dimensions.append(line.split(" "))    # Dimensions of each plate
    for dim in dimensions:
        dim[0] = int(dim[0])
        dim[1] = int(dim[1])
    instance = Instance(width, n, dimensions)
    return instance


def solve(instance):
    solver = Solver()
    
    solver.set('timeout', 300 * 1000)

    # Variable Initalization

    x_dims=[]
    y_dims=[]

    for x_dim,y_dim in instance.dimensions:
        x_dims.append(x_dim)
        y_dims.append(y_dim )

    max_height = math.ceil(sum(y_dims)/(instance.width//max(x_dims)))

    print("Width of the plate: ")
    print(instance.width)
    print("Max height of the plate: ")
    print(max_height)


    # Plate width * max_height. Each position has n (nr of blocks) boolean values
    boolean_plate = [[[Bool("coord_b"+str(b+1)+"_x"+str(x)+"_y"+str(y)) for b in range(instance.n)] for x in range(instance.width) ] for y in range(max_height)]  
    #print(boolean_plate)

    # Constraints

    # Blocks must not overlap together

    for y in range(max_height):
        for x in range(instance.width):
            solver.add(at_most_one(boolean_plate[y][x], "unique_{}_{}".format(y,x)))     # Only one True on each plate cell

        
    # Rotation
    
    rotation = [Bool("block_{}".format(i)) for i in range(instance.n)]

    not_rotated_blocks_positions=[]
    rotated_blocks_positions=[]
    final=[]

    for b in range(instance.n):
        x_dims, y_dims = y_dims, x_dims

        blocks_positions = []
        for y in range(max_height - y_dims[b]+1):
            for x in range(instance.width - x_dims[b]+1):
                blocks_positions.append(And([boolean_plate[j][i][b] for j in range(y, y+y_dims[b]) for i in range(x,x+x_dims[b])]))
        
        not_rotated_blocks_positions = And(Not(rotation[b]), And(exactly_one(blocks_positions,"position_{}".format(b))))
        
        x_dims, y_dims = y_dims, x_dims
    
        blocks_positions2 = []
        for y in range(max_height - y_dims[b]+1):
            for x in range(instance.width - x_dims[b]+1):
                blocks_positions2.append(And([boolean_plate[j][i][b] for j in range(y, y+y_dims[b]) for i in range(x,x+x_dims[b])]))
        
        rotated_blocks_positions = And(rotation[b], And(exactly_one(blocks_positions2,"position2_{}".format(b))))
        
        final.append(exactly_one([not_rotated_blocks_positions,rotated_blocks_positions],"position3_{}".format(b)))
        
    solver.add(final)
    
    
    # One hot encoding of height 

    solution_height = [Bool("h"+str(height)) for height in range(max_height)]   # [h1,h2,h3,h4,...]

    solver.add(exactly_one(solution_height, "one"))    # Only one value True on solution_height
    
    for y in range(max_height):
        solver.add(solution_height[y] == And(at_least_one(list(np.ravel(boolean_plate[y])))
                                             , exactly_zero(list(np.ravel(boolean_plate[y+1:])))))
        
        
    solver.set('timeout', 300 * 1000)

    satisfiable = False
    
    for i in range(max_height):
        solver.push()
        solver.add(solution_height[i])
        if solver.check() == sat:
            model = solver.model()
            satisfiable = True
            break
        solver.pop()
    if satisfiable: 
        print("Satisfiable")
        print("Solution - plate height:")
        print(i+1)
    else: print("Not satisfiable")

    return solver, boolean_plate, satisfiable

def main():
    instance_file = "../../instances/ins-1.txt"
    instance = read_file(instance_file)

    start = time.time()
    solver,boolean_plate, satisfiable = solve(instance)
    end = time.time()

    print("{:.2f}".format(end - start) + " seconds")
    
    for x in range(1,40):
        
        instance_file = "../../instances/ins-{}.txt".format(x)
        instance = read_file(instance_file)
        
        start = time.time()
        solver,boolean_plate, satisfiable = solve(instance)
        end = time.time()
        
        print("Instance",x)
        print("\t{:.2f}".format(end - start) + " seconds")
        
        if satisfiable:
            with open('insr{}SAT_rot.txt'.format(x), 'w') as myfile:
                myfile.write('time='+'{:.3f}'.format(end - start))
                myfile.close()
        else:
            with open('insr{}SAT_rot.txt'.format(x), 'w') as myfile:
                myfile.write("time=0.000")
                myfile.close()


if __name__ == '__main__':
    main()

Width of the plate: 
8
Max height of the plate: 
16
Satisfiable
Solution - plate height:
8
1.53 seconds
Width of the plate: 
8
Max height of the plate: 
16
Satisfiable
Solution - plate height:
8
Instance 1
	1.62 seconds
Width of the plate: 
9
Max height of the plate: 
9
Satisfiable
Solution - plate height:
9
Instance 2
	1.08 seconds
Width of the plate: 
10
Max height of the plate: 
15
Satisfiable
Solution - plate height:
10
Instance 3
	5.28 seconds
Width of the plate: 
11
Max height of the plate: 
29
Satisfiable
Solution - plate height:
11
Instance 4
	90.42 seconds
Width of the plate: 
12
Max height of the plate: 
23
Satisfiable
Solution - plate height:
21
Instance 5
	69.29 seconds
Width of the plate: 
13
Max height of the plate: 
46


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 2878, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-bed4a5256dae>", line 190, in <module>
    main()
  File "<ipython-input-3-bed4a5256dae>", line 173, in main
    solver,boolean_plate, satisfiable = solve(instance)
  File "<ipython-input-3-bed4a5256dae>", line 108, in solve
    blocks_positions.append(And([boolean_plate[j][i][b] for j in range(y, y+y_dims[b]) for i in range(x,x+x_dims[b])]))
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5/site-packages/z3/z3.py", line 1841, in And
    args = _coerce_expr_list(args, ctx)
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5/site-packages/z3/z3.py", line 1206, in _coerce_expr_list
    s = _reduce(_coerce_expr_merge, alist, None)
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5/site-packages

KeyboardInterrupt: 

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5/site-packages/zmq/eventloop/zmqstream.py", line 432, in _run_callback
    callback(*args, **kwargs)
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5/site-packages/tornado/stack_context.py", line 300, in null_wrapper
    return fn(*args, **kwargs)
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 283, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 233, in dispatch_shell
    handler(stream, idents, msg)
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 421, in execute_request
    self._abort_queues()
  File "/Users/mazeyarmoeini/opt/anaconda3/envs/VLSI/lib/python3.5

In [8]:
x = Bool("x")
y = Bool("y")
z = Bool("z")
at_most_one([x,y,z])

TypeError: at_most_one() missing 1 required positional argument: 'name'

In [77]:
x = [1,2,3]
x[1:3]

[2, 3]

In [58]:
def at_least_one(bool_vars):
    return Or(bool_vars)

def at_most_one(bool_vars, name):
    constraints = []
    n = len(bool_vars)
    s = [Bool(f"s_{name}_{i}") for i in range(n - 1)]
    constraints.append(Or(Not(bool_vars[0]), s[0]))
    constraints.append(Or(Not(bool_vars[n-1]), Not(s[n-2])))
    for i in range(1, n - 1):
        constraints.append(Or(Not(bool_vars[i]), s[i]))
        constraints.append(Or(Not(bool_vars[i]), Not(s[i-1])))
        constraints.append(Or(Not(s[i-1]), s[i]))
    return And(constraints)

def exactly_one(bool_vars, name):
    return And(at_most_one(bool_vars, name),at_least_one(bool_vars))


a = And([True,False, And(False)])
b= And([True, True, And(False)])
res = exactly_one([a,b],"ok")
print(res)

Z3Exception: True, False or Z3 Boolean expression expected. Received [And(True, False, And(False))] of type <class 'list'>